### Notebook for concept detection in neural network

In [1]:
%load_ext autoreload
%autoreload 2

In [29]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

import concepts
import env
from policy import ActorCriticNet

model_name = "net"
session_name = "starship"
board_size = 5
board_name = f'{board_size}x{board_size}'

agents_to_sample = [0, 150, 300, 450, 600, 750, 900, 1050, 1200, 1500, 1800]

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = concepts.play_center_in_opening

CONCEPT_NAME = concepts.play_center_in_opening.__name__

CASES_TO_SAMPLE = 1000 # 25000


In [30]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [32]:
def play_match(agents: list[ActorCriticNet], board_size, concept_function):
    go_env = env.GoEnv(board_size)
    state = go_env.reset()

    player_to_start = 1 if np.random.random() > 0.5 else 0
    current_player = player_to_start

    total_moves = board_size * board_size * 4
    moves = 0
    random_moves = False

    positive_cases = []
    negative_cases = []

    sample_ratio = 0.2
    alpha = 0.2 # The higher, the more unique games. 0 = argmax, 1 = distribution 

    game_over = False

    while not game_over:
        if moves == total_moves:
            break
        # Remove array index 3 and 5 from the current state making it an shape of (4, 5, 5)
        state_copy = np.delete(state, [3, 5], axis=0)
        
        if np.random.random() < sample_ratio:
            pos = concept_function(state)
            if pos:
                positive_cases.append(state_copy)
            elif not pos:
                negative_cases.append(state_copy)

        if random_moves:
            action = go_env.uniform_random_action()
        else:
            action = agents[current_player].best_action(state, alpha=alpha)

        state, _, game_over, _ = go_env.step(action)
        moves += 1

        current_player = (current_player + 1) % 2
    
    return positive_cases, negative_cases

positive_cases = []
negative_cases = []

positive_bar = tqdm(total=CASES_TO_SAMPLE, desc="Positive cases")

while len(positive_cases) < CASES_TO_SAMPLE or len(negative_cases) < CASES_TO_SAMPLE:
    for i in range(len(agents)):
        for j in range(i + 1, len(agents)):
            pos, neg = play_match([agents[i], agents[j]], board_size, CONCEPT_FUNC)

            positive_cases.extend(pos)
            negative_cases.extend(neg)

            positive_bar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_SAMPLE]
negative_cases = negative_cases[:CASES_TO_SAMPLE]

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

Positive cases:   1%|          | 10/1000 [00:03<03:20,  4.94it/s]

5.0


Positive cases:   2%|▏         | 20/1000 [00:05<03:16,  4.98it/s]

7.0
8.0


Positive cases:   2%|▏         | 23/1000 [00:05<02:41,  6.05it/s]

...

Positive cases: 1007it [05:37,  4.46it/s]                          

5.0


Positive cases: 1012it [05:39,  3.32it/s]

6.0
5.0


Positive cases: 1014it [05:39,  4.38it/s]

In [33]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (1000, 4, 5, 5)
Negative cases:  (1000, 4, 5, 5)


In [5]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [34]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  736
Duplicate count in negative:  38


In [35]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print()

Positive case 0:
[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]]

Positive case 1:
[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]]

Positive case 2:
[[[0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.

In [36]:
# Positions to consider are 80% of the total positions
POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0] * 2)
print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
#POSITIONS_TO_CONSIDER = 4000 #40000
VALIDATION_POSITIONS = 10000 #10000

BINARY = True

Positions to consider: 1600


In [37]:
# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

# Use the regression
score = concepts.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
    # Test if epoch folder exists
    if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
            os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name, file))

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name), exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, name, random_suffix), "w") as f:
    json.dump(score, f)

Epoch 1/50


50/50 [==============================] - 1s 5ms/step - loss: 0.8897 - val_loss: 0.8613
Epoch 2/50
50/50 [==============================] - 0s 2ms/step - loss: 0.8666 - val_loss: 0.8429
Epoch 3/50
50/50 [==============================] - 0s 2ms/step - loss: 0.8459 - val_loss: 0.8270
Epoch 4/50
50/50 [==============================] - 0s 3ms/step - loss: 0.8278 - val_loss: 0.8128
Epoch 5/50
50/50 [==============================] - 0s 2ms/step - loss: 0.8115 - val_loss: 0.7992
Epoch 6/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7964 - val_loss: 0.7870
Epoch 7/50
50/50 [==============================] - 0s 2ms/step - loss: 0.7825 - val_loss: 0.7757
Epoch 8/50
50/50 [==============================] - 0s 2ms/step - loss: 0.7696 - val_loss: 0.7650
Epoch 9/50
50/50 [==============================] - 0s 2ms/step - loss: 0.7576 - val_loss: 0.7548
Epoch 10/50
50/50 [==============================] - 0s 3ms/step - loss: 0.7463 - val_loss: 0.7455
Epoch 11/50
50/50 [===========

In [38]:
epochs_to_look_at = [0, 150, 300, 450, 600, 750, 900, 1050, 1200, 1500, 1800]

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = concepts.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)


Getting activation outputs: 100%|██████████| 63/63 [00:00<00:00, 224.06it/s]


Performing regression for layer 0
Epoch 1/50
50/50 [==============================] - 1s 8ms/step - loss: 0.9664 - val_loss: 0.9531
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9408 - val_loss: 0.9275
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9169 - val_loss: 0.9037
Epoch 4/50
50/50 [==============================] - 0s 2ms/step - loss: 0.8944 - val_loss: 0.8811
Epoch 5/50
50/50 [==============================] - 0s 2ms/step - loss: 0.8729 - val_loss: 0.8596
Epoch 6/50
50/50 [==============================] - 0s 2ms/step - loss: 0.8525 - val_loss: 0.8396
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8333 - val_loss: 0.8205
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8153 - val_loss: 0.8027
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7984 - val_loss: 0.7863
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7824 - va

Getting activation outputs: 100%|██████████| 63/63 [00:00<00:00, 287.96it/s]

Performing regression for layer 0
Epoch 1/50


50/50 [==============================] - 0s 3ms/step - loss: 1.1247 - val_loss: 0.9998
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9755 - val_loss: 0.9131
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9004 - val_loss: 0.8657
Epoch 4/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8546 - val_loss: 0.8315
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8201 - val_loss: 0.8038
Epoch 6/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7914 - val_loss: 0.7796
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7665 - val_loss: 0.7579
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7443 - val_loss: 0.7387
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7241 - val_loss: 0.7206
Epoch 10/50
50/50 [==============================] - 0s 2ms/step - loss: 0.7059 - val_loss: 0.7043
Epoch 11/50
50/50 [===========

...

Getting activation outputs: 100%|██████████| 63/63 [00:00<00:00, 287.94it/s]

Performing regression for layer 0
Epoch 1/50


50/50 [==============================] - 0s 3ms/step - loss: 0.9614 - val_loss: 0.9265
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8740 - val_loss: 0.8620
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8141 - val_loss: 0.8126
Epoch 4/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7675 - val_loss: 0.7731
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7301 - val_loss: 0.7415
Epoch 6/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6997 - val_loss: 0.7159
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6746 - val_loss: 0.6935
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6526 - val_loss: 0.6744
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6339 - val_loss: 0.6577
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6170 - val_loss: 0.6428
Epoch 11/50
50/50 [===========

Getting activation outputs: 100%|██████████| 63/63 [00:00<00:00, 198.34it/s]


Performing regression for layer 0
Epoch 1/50
50/50 [==============================] - 0s 3ms/step - loss: 1.0878 - val_loss: 1.0189
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9798 - val_loss: 0.9274
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9065 - val_loss: 0.8574
Epoch 4/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8487 - val_loss: 0.8020
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8016 - val_loss: 0.7580
Epoch 6/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7635 - val_loss: 0.7229
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7318 - val_loss: 0.6940
Epoch 8/50
50/50 [==============================] - 0s 2ms/step - loss: 0.7052 - val_loss: 0.6696
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6822 - val_loss: 0.6487
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6619 - va